In [537]:
#Difference Numbers, 3 Targets
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [538]:
walmart = pd.read_csv('walmartdata.csv')
walmart.head()

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68


In [539]:
totalave = walmart.groupby(["Store"])["Weekly_Sales"].mean()
fuelave = walmart.groupby(["Store"])["Fuel_Price"].mean()
unemave = walmart.groupby(["Store"])["Unemployment"].mean()
cpiave = walmart.groupby(["Store"])["CPI"].mean()
tempave = walmart.groupby(["Store"])["Temperature_C"].mean()

In [540]:
for i in range(len(walmart)):
    walmart.loc[i, "Ave_Sales"] = totalave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Fuel"] = fuelave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Temp"] = tempave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_Unem"] = unemave[walmart.loc[i, "Store"]]
    walmart.loc[i, "Ave_CPI"] = cpiave[walmart.loc[i, "Store"]]
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,Week,Year,Temperature_C,Holiday_Name,Weekly_Sales,Ave_Sales,Ave_Fuel,Ave_Temp,Ave_Unem,Ave_CPI
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,5,2010,6.0,No Holiday,1643690.90,1.555264e+06,3.219790,20.167832,7.610420,215.996891
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,6,2010,4.0,No Holiday,1641957.44,1.555264e+06,3.219790,20.167832,7.610420,215.996891
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,7,2010,4.0,No Holiday,1611968.17,1.555264e+06,3.219790,20.167832,7.610420,215.996891
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,8,2010,8.0,No Holiday,1409727.59,1.555264e+06,3.219790,20.167832,7.610420,215.996891
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,9,2010,8.0,No Holiday,1554806.68,1.555264e+06,3.219790,20.167832,7.610420,215.996891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6434,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,39,2012,18.0,No Holiday,713173.95,7.859814e+05,3.417343,14.342657,8.648748,186.285678
6435,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,40,2012,18.0,No Holiday,733455.07,7.859814e+05,3.417343,14.342657,8.648748,186.285678
6436,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,41,2012,12.0,No Holiday,734464.36,7.859814e+05,3.417343,14.342657,8.648748,186.285678
6437,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,42,2012,14.0,No Holiday,718125.53,7.859814e+05,3.417343,14.342657,8.648748,186.285678


In [541]:
walmart['Fuel_Diff'] = (walmart['Fuel_Price'] - walmart["Ave_Fuel"])
walmart['Temp_Diff'] = (walmart['Temperature_C'] - walmart["Ave_Temp"])
walmart['Unem_Diff'] = (walmart['Unemployment'] - walmart["Ave_Unem"])
walmart['CPI_Diff'] = (walmart['CPI'] - walmart["Ave_CPI"])
walmart

,Unnamed: 0,ID,Store,Start_of_Week,Week_Date,Fuel_Price,CPI,Unemployment,Type,Size,...,Weekly_Sales,Ave_Sales,Ave_Fuel,Ave_Temp,Ave_Unem,Ave_CPI,Fuel_Diff,Temp_Diff,Unem_Diff,CPI_Diff
0,0,0,1,2010-01-30,2010-02-05,2.57,211.0964,8.106,A,151315,...,1643690.90,1.555264e+06,3.219790,20.167832,7.610420,215.996891,-0.649790,-14.167832,0.495580,-4.900491
1,1,1,1,2010-02-06,2010-02-12,2.55,211.2422,8.106,A,151315,...,1641957.44,1.555264e+06,3.219790,20.167832,7.610420,215.996891,-0.669790,-16.167832,0.495580,-4.754691
2,2,2,1,2010-02-13,2010-02-19,2.51,211.2891,8.106,A,151315,...,1611968.17,1.555264e+06,3.219790,20.167832,7.610420,215.996891,-0.709790,-16.167832,0.495580,-4.707791
3,3,3,1,2010-02-20,2010-02-26,2.56,211.3196,8.106,A,151315,...,1409727.59,1.555264e+06,3.219790,20.167832,7.610420,215.996891,-0.659790,-12.167832,0.495580,-4.677291
4,4,4,1,2010-02-27,2010-03-05,2.62,211.3501,8.106,A,151315,...,1554806.68,1.555264e+06,3.219790,20.167832,7.610420,215.996891,-0.599790,-12.167832,0.495580,-4.646791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6434,6434,6434,45,2012-09-22,2012-09-28,4.00,192.0136,8.684,B,118221,...,713173.95,7.859814e+05,3.417343,14.342657,8.648748,186.285678,0.582657,3.657343,0.035252,5.727922
6435,6435,6435,45,2012-09-29,2012-10-05,3.98,192.1704,8.667,B,118221,...,733455.07,7.859814e+05,3.417343,14.342657,8.648748,186.285678,0.562657,3.657343,0.018252,5.884722
6436,6436,6436,45,2012-10-06,2012-10-12,4.00,192.3273,8.667,B,118221,...,734464.36,7.859814e+05,3.417343,14.342657,8.648748,186.285678,0.582657,-2.342657,0.018252,6.041622
6437,6437,6437,45,2012-10-13,2012-10-19,3.97,192.3309,8.667,B,118221,...,718125.53,7.859814e+05,3.417343,14.342657,8.648748,186.285678,0.552657,-0.342657,0.018252,6.045222


In [542]:
walmart = walmart.filter(items=['Fuel_Diff', "CPI_Diff", "Unem_Diff", "Temp_Diff", "Weekly_Sales","Ave_Sales"])

In [543]:
#walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Weekly_Sales","Ave_Sales"])

In [544]:
walmart.loc[walmart['Weekly_Sales'] < (0.85 * walmart["Ave_Sales"]), 'Label'] = 'Below' 
walmart.loc[walmart['Weekly_Sales'] > (1.15 * walmart["Ave_Sales"]), 'Label'] = 'Above' 
walmart.loc[(walmart['Weekly_Sales'] > (0.85 * walmart["Ave_Sales"])) & (walmart['Weekly_Sales'] < (1.15 * walmart["Ave_Sales"])), 'Label'] = 'Average'

In [545]:
#walmart = walmart.filter(items=['Fuel_Price', "CPI", "Unemployment", "Temperature_C", "Label"])

In [546]:
walmart = walmart.filter(items=['Fuel_Diff', "CPI_Diff", "Unem_Diff", "Temp_Diff", "Label"])

In [547]:
walmart["Label"].value_counts()

Average    5601
Above       500
Below       338
Name: Label, dtype: int64

In [548]:
X = walmart.drop("Label", axis=1)
y = walmart["Label"]
print(X.shape, y.shape)

(6439, 4) (6439,)


In [549]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [550]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

# Scale and train x value datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [551]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [519]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model

LogisticRegression(random_state=42)

In [375]:
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(random_state=42)

In [376]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [377]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'solver': ['newton-cg', 'liblinear', 'sag', 'saga'],
             'max_iter':[200, 500, 1000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [378]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=newton-cg .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] . C=1, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=1, max_iter=200, solver=newton-cg .............................
[CV] . C=1, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.2s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=1, max_iter=200, solver=liblinear .............................
[CV] . C=1, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=1, max_iter=200, solver=sag ...................................
[CV] .

[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=newton-cg .............................
[CV] . C=5, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] . C=5, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=5, max_iter=200, solver=liblinear .............................
[CV] .

[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=newton-cg ............................
[CV]  C=10, max_iter=200, solver=newton-cg, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  C=10, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=10, max_iter=200, solver=liblinear ............................
[CV]  

[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.871, total=   0.1s
[CV] C=50, max_iter=200, solver=newton-cg ............................
[CV]  C=50, max_iter=200, solver=newton-cg, score=0.872, total=   0.1s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.872, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  C=50, max_iter=200, solver=liblinear, score=0.871, total=   0.0s
[CV] C=50, max_iter=200, solver=liblinear ............................
[CV]  

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed:    6.3s finished


GridSearchCV(estimator=LogisticRegression(random_state=42),
             param_grid={'C': [1, 5, 10, 50], 'max_iter': [200, 500, 1000],
                         'solver': ['newton-cg', 'liblinear', 'sag', 'saga']},
             verbose=3)

In [379]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'max_iter': 200, 'solver': 'newton-cg'}
0.8709878887351291


In [382]:
logreg = LogisticRegression(C=1, max_iter=200, solver= 'newton-cg', random_state=42)
logreg.fit(X_train_scaled, encoded_y_train)
print(f"Training Data Score: {logreg.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {logreg.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8709877821495133
Testing Data Score: 0.8664596273291926


In [552]:
target = walmart["Label"]
target_names = ["Above", "Average", "Below"]

In [553]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf = rf.fit(X_train_scaled, encoded_y_train)
rf.score(X_test_scaled, encoded_y_test)

0.8919254658385093

In [554]:
feature_names = walmart.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2827384801276348, 'Fuel_Diff'),
 (0.26451415541645934, 'Temp_Diff'),
 (0.25873641347977583, 'CPI_Diff'),
 (0.19401095097612997, 'Unem_Diff')]

In [385]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(clf, param_grid, verbose=3)

In [386]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [387]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [555]:
ranfor = RandomForestClassifier(n_estimators=500, random_state=42)
ranfor = ranfor.fit(X_train_scaled, encoded_y_train)
ranfor.score(X_test_scaled, encoded_y_test)

0.8913043478260869

In [556]:
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train_scaled, encoded_y_train)
clf.score(X_test_scaled, encoded_y_test)

0.8633540372670807

In [436]:
feature_names = walmart.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.36486851439318435, 'CPI'),
 (0.27004417922651425, 'Fuel_Price'),
 (0.182869784976228, 'Unemployment'),
 (0.1822175214040735, 'Temperature_C')]

In [391]:
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
             'class_weight': ['None', 'balanced']}
grid = GridSearchCV(clf, param_grid, verbose=3)

In [392]:
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=best ................
[CV]  class_weight=None, criterion=gini, splitter=best, score=nan, total=   0.0s
[CV] class_weight=None, criterion=gini, splitter=random ..............
[CV]  class_weight=None, criterion=gini, splitter=random, score=nan, total=   0.0s
[CV] class_weight=None, cr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 199, in fit
    expanded_class_weight = compute_sample_weight(
  File "/Users/ronaldclarke/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 73, in inner_f
    return f(**kwargs)
  File "/Users/ronal

[CV]  class_weight=balanced, criterion=entropy, splitter=best, score=0.851, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.827, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.836, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.839, total=   0.0s
[CV] class_weight=balanced, criterion=entropy, splitter=random .......
[CV]  class_weight=balanced, criterion=entropy, splitter=random, score=0.829, total=   0.0s


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.2s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'class_weight': ['None', 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [393]:
print(grid.best_params_)
print(grid.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'splitter': 'best'}
0.8482092706422512


In [557]:
dectre = tree.DecisionTreeClassifier(random_state=42)
dectre = dectre.fit(X_train_scaled, encoded_y_train)
dectre.score(X_test_scaled, encoded_y_test)

0.8633540372670807

In [558]:
dectre_predictions = dectre.predict(X_test_scaled)

In [559]:
ranfor_predictions = ranfor.predict(X_test_scaled)

In [560]:
print(classification_report(encoded_y_test, ranfor_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.74      0.49      0.59       134
     Average       0.91      0.97      0.94      1395
       Below       0.48      0.25      0.33        81

    accuracy                           0.89      1610
   macro avg       0.71      0.57      0.62      1610
weighted avg       0.88      0.89      0.88      1610



In [561]:
print(classification_report(encoded_y_test, dectre_predictions,
                            target_names= target_names))

              precision    recall  f1-score   support

       Above       0.56      0.56      0.56       134
     Average       0.92      0.93      0.92      1395
       Below       0.33      0.28      0.30        81

    accuracy                           0.86      1610
   macro avg       0.60      0.59      0.60      1610
weighted avg       0.86      0.86      0.86      1610



In [599]:
predix = pd.read_csv('prophet_predictions.csv')
predix = predix.drop(['Unnamed: 0'], axis=1)
predix.head()

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06


In [600]:
for i in range(len(predix)):
    predix.loc[i, "Ave_Fuel"] = fuelave[predix.loc[i, "Store"]]
    predix.loc[i, "Ave_CPI"] = cpiave[predix.loc[i, "Store"]]
    predix.loc[i, "Ave_Temp"] = tempave[predix.loc[i, "Store"]]
    predix.loc[i, "Ave_Unem"] = unemave[predix.loc[i, "Store"]]
predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Fuel,Ave_CPI,Ave_Temp,Ave_Unem
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,3.219790,215.996891,20.167832,7.610420
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,3.219790,215.996891,20.167832,7.610420
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,3.219790,215.996891,20.167832,7.610420
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,3.219790,215.996891,20.167832,7.610420
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,3.219790,215.996891,20.167832,7.610420
...,...,...,...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,3.417343,186.285678,14.342657,8.648748
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,3.417343,186.285678,14.342657,8.648748
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,3.417343,186.285678,14.342657,8.648748
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,3.417343,186.285678,14.342657,8.648748


In [601]:
predix['Fuel_Diff'] = (predix['Fuel_Price'] - predix["Ave_Fuel"])
predix['Temp_Diff'] = (predix['Temperature_C'] - predix["Ave_Temp"])
predix['Unem_Diff'] = (predix['Unemployment'] - predix["Ave_Unem"])
predix['CPI_Diff'] = (predix['CPI'] - predix["Ave_CPI"])
predix

,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Fuel,Ave_CPI,Ave_Temp,Ave_Unem,Fuel_Diff,Temp_Diff,Unem_Diff,CPI_Diff
0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,3.219790,215.996891,20.167832,7.610420,0.267649,-2.507938,-0.982450,7.464254
1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,3.219790,215.996891,20.167832,7.610420,0.252542,-3.718840,-0.975968,7.446281
2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,3.219790,215.996891,20.167832,7.610420,0.236833,-5.063254,-0.986453,7.412851
3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,3.219790,215.996891,20.167832,7.610420,0.221783,-6.547066,-1.017996,7.375718
4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,3.219790,215.996891,20.167832,7.610420,0.203749,-8.139037,-1.053631,7.361601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,3.417343,186.285678,14.342657,8.648748,0.975846,2.227712,0.229388,8.850462
2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,3.417343,186.285678,14.342657,8.648748,0.983458,1.028496,0.223930,8.891549
2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,3.417343,186.285678,14.342657,8.648748,0.977529,-0.932962,0.228578,8.875975
2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,3.417343,186.285678,14.342657,8.648748,0.963550,-3.014594,0.239096,8.829505


In [602]:
predix = predix.filter(items=['Date', 'Store', 'Fuel_Diff', "CPI_Diff", "Unem_Diff", "Temp_Diff"])

In [603]:
predix
X = predix.drop("Date", axis=1)
X = X.drop("Store", axis=1)
X

,Fuel_Diff,CPI_Diff,Unem_Diff,Temp_Diff
0,0.267649,7.464254,-0.982450,-2.507938
1,0.252542,7.446281,-0.975968,-3.718840
2,0.236833,7.412851,-0.986453,-5.063254
3,0.221783,7.375718,-1.017996,-6.547066
4,0.203749,7.361601,-1.053631,-8.139037
...,...,...,...,...
2470,0.975846,8.850462,0.229388,2.227712
2471,0.983458,8.891549,0.223930,1.028496
2472,0.977529,8.875975,0.228578,-0.932962
2473,0.963550,8.829505,0.239096,-3.014594


In [604]:
predix_scaled = X_scaler.transform(X)

In [605]:
ranfor_labels = ranfor.predict(predix_scaled)

In [606]:
ranfor_labels = label_encoder.inverse_transform(ranfor_labels)
ranfor_labels

array(['Average', 'Average', 'Average', ..., 'Average', 'Average',
       'Average'], dtype=object)

In [607]:
ranfor_predix = predix
ranfor_predix["Label"] = ranfor_labels
ranfor_predix

,Date,Store,Fuel_Diff,CPI_Diff,Unem_Diff,Temp_Diff,Label
0,2012-10-21,1,0.267649,7.464254,-0.982450,-2.507938,Average
1,2012-10-28,1,0.252542,7.446281,-0.975968,-3.718840,Average
2,2012-11-04,1,0.236833,7.412851,-0.986453,-5.063254,Average
3,2012-11-11,1,0.221783,7.375718,-1.017996,-6.547066,Average
4,2012-11-18,1,0.203749,7.361601,-1.053631,-8.139037,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,0.975846,8.850462,0.229388,2.227712,Average
2471,2013-10-13,45,0.983458,8.891549,0.223930,1.028496,Average
2472,2013-10-20,45,0.977529,8.875975,0.228578,-0.932962,Average
2473,2013-10-27,45,0.963550,8.829505,0.239096,-3.014594,Average


In [608]:
ranfor_predix["Label"].value_counts()

Average    2399
Below        71
Above         5
Name: Label, dtype: int64

In [588]:
dectre_labels = dectre.predict(predix_scaled)

In [589]:
dectre_labels = label_encoder.inverse_transform(dectre_labels)
dectre_labels

array(['Average', 'Average', 'Average', ..., 'Below', 'Below', 'Below'],
      dtype=object)

In [590]:
dectre_predix = predix
dectre_predix["Label"] = dectre_labels
dectre_predix

,Date,Store,Fuel_Diff,CPI_Diff,Unem_Diff,Temp_Diff,Label
0,2012-10-21,1,0.267649,7.464254,-0.982450,-2.507938,Average
1,2012-10-28,1,0.252542,7.446281,-0.975968,-3.718840,Average
2,2012-11-04,1,0.236833,7.412851,-0.986453,-5.063254,Average
3,2012-11-11,1,0.221783,7.375718,-1.017996,-6.547066,Average
4,2012-11-18,1,0.203749,7.361601,-1.053631,-8.139037,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,0.975846,8.850462,0.229388,2.227712,Below
2471,2013-10-13,45,0.983458,8.891549,0.223930,1.028496,Below
2472,2013-10-20,45,0.977529,8.875975,0.228578,-0.932962,Below
2473,2013-10-27,45,0.963550,8.829505,0.239096,-3.014594,Below


In [591]:
dectre_predix["Label"].value_counts()

Average    1987
Below       457
Above        31
Name: Label, dtype: int64

In [592]:
predix

,Date,Store,Fuel_Diff,CPI_Diff,Unem_Diff,Temp_Diff,Label
0,2012-10-21,1,0.267649,7.464254,-0.982450,-2.507938,Average
1,2012-10-28,1,0.252542,7.446281,-0.975968,-3.718840,Average
2,2012-11-04,1,0.236833,7.412851,-0.986453,-5.063254,Average
3,2012-11-11,1,0.221783,7.375718,-1.017996,-6.547066,Average
4,2012-11-18,1,0.203749,7.361601,-1.053631,-8.139037,Average
...,...,...,...,...,...,...,...
2470,2013-10-06,45,0.975846,8.850462,0.229388,2.227712,Below
2471,2013-10-13,45,0.983458,8.891549,0.223930,1.028496,Below
2472,2013-10-20,45,0.977529,8.875975,0.228578,-0.932962,Below
2473,2013-10-27,45,0.963550,8.829505,0.239096,-3.014594,Below


In [593]:
predix = pd.read_csv('prophet_predictions.csv')

In [594]:
for i in range(len(predix)):
    predix.loc[i, "Ave_Sales"] = totalave[predix.loc[i, "Store"]]
predix

,Unnamed: 0,Date,Store,Fuel_Price,CPI,Unemployment,Temperature_C,Weekly_Sales,Ave_Sales
0,0,2012-10-21,1,3.487440,223.461145,6.627970,17.659895,1.529256e+06,1.555264e+06
1,1,2012-10-28,1,3.472333,223.443172,6.634452,16.448992,1.587566e+06,1.555264e+06
2,2,2012-11-04,1,3.456623,223.409742,6.623966,15.104578,1.678286e+06,1.555264e+06
3,3,2012-11-11,1,3.441573,223.372609,6.592424,13.620766,1.742059e+06,1.555264e+06
4,4,2012-11-18,1,3.423539,223.358492,6.556789,12.028795,1.765627e+06,1.555264e+06
...,...,...,...,...,...,...,...,...,...
2470,2470,2013-10-06,45,4.393188,195.136141,8.878137,16.570370,7.524903e+05,7.859814e+05
2471,2471,2013-10-13,45,4.400800,195.177228,8.872678,15.371153,7.420480e+05,7.859814e+05
2472,2472,2013-10-20,45,4.394871,195.161653,8.877326,13.409695,7.451909e+05,7.859814e+05
2473,2473,2013-10-27,45,4.380892,195.115184,8.887845,11.328064,7.835365e+05,7.859814e+05


In [595]:
predix.loc[predix['Weekly_Sales'] < (0.85 * predix["Ave_Sales"]), 'Label'] = 'Below' 
predix.loc[predix['Weekly_Sales'] > (1.15 * predix["Ave_Sales"]), 'Label'] = 'Above' 
predix.loc[(predix['Weekly_Sales'] > (0.85 * predix["Ave_Sales"])) & (predix['Weekly_Sales'] < (1.15 * predix["Ave_Sales"])), 'Label'] = 'Average'
proph_predix = predix

In [596]:
proph_predix["Label"].value_counts()

Average    1888
Above       404
Below       183
Name: Label, dtype: int64

In [597]:
dectre_predix["Label"].value_counts()

Average    1987
Below       457
Above        31
Name: Label, dtype: int64

In [609]:
ranfor_predix["Label"].value_counts()

Average    2399
Below        71
Above         5
Name: Label, dtype: int64